In [18]:
import json

In [19]:
#open json
f = open('wix-webhooks-event-conversion.json')
data = json.load(f)

In [20]:
type(data)

dict

In [21]:
json.loads(data['body'].replace("\\"," "))['data']

{'plan_valid_until': "Fino all 'annullamento",
 'site_email': 'contatti@weavingtech.net',
 'plan_order_id': 'c49ae422-7102-4ff4-95b9-ac66e93cde39',
 'context': {'metaSiteId': 'be57c73f-a509-4490-a1f5-a45da5cc5043',
  'activationId': 'f3c34a8a-0f9d-4592-b313-9abba689d129'},
 '_context': {'activation': {'id': 'f3c34a8a-0f9d-4592-b313-9abba689d129'},
  'configuration': {'id': '0655f07d-6608-4fc6-9bfa-cfc706a9e2bd'},
  'app': {'id': '1522827f-c56c-a5c9-2ac9-00f9e6ae12d3'},
  'action': {'id': 'f96325e4-7ca7-d3e1-47f6-3598f82f5e9c'},
  'trigger': {'key': 'wix_pricing_plans-plan_purchased'}},
 'plan_description': 'Per i professionisti',
 'contact': {'name': {'first': 'Rosario', 'last': 'Laface'},
  'email': 'onslaught89@gmail.com',
  'phones': [{'tag': 'UNTAGGED',
    'formattedPhone': '3402396507',
    'id': 'b8c73aa4-0799-47af-92bb-1f331720c824',
    'primary': True,
    'phone': '3402396507'}],
  'address': {'addressLine': 'via San Vigilio 33',
   'formattedAddress': 'via San Vigilio 33',


In [22]:
json.loads(data['body'].replace("\\"," "))['data']['plan_title']

'Base'

In [23]:
email = json.loads(data['body'].replace("\\"," "))['data']['contact']['email']
email

'onslaught89@gmail.com'

In [24]:
first_name = json.loads(data['body'].replace("\\"," "))['data']['contact']['name']['first']
first_name

'Rosario'

In [25]:
last_name = json.loads(data['body'].replace("\\"," "))['data']['contact']['name']['last']
last_name

'Laface'

In [26]:
phone = json.loads(data['body'].replace("\\"," "))['data']['contact']['phones'][0]['phone']
phone

'3402396507'

In [27]:
address = json.loads(data['body'].replace("\\"," "))['data']['contact']['address']['addressLine']
address

'via San Vigilio 33'

In [28]:
from dotenv import load_dotenv
import os
load_dotenv('.env', override=True)
import boto3

In [29]:
cognito_client = boto3.client('cognito-idp', region_name=os.getenv('AWS_REGION'))  
dynamodb_client = boto3.resource('dynamodb', region_name=os.getenv('AWS_REGION'))
table = dynamodb_client.Table(os.getenv('DYNAMODB_TABLE'))

In [30]:
def add_to_dynamodb(email, tier):
    '''
    Update the tier of the recently added user.
    ''' 
    key = {"email": email}
    update_expression = "SET tier = :new_value"
    expression_values = {":new_value": tier}
    
    # try:
    response = table.update_item(
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeValues=expression_values,
    )
    print("Item updated successfully:", response)
    # except Exception as e:
    #     print("Error updating item:", e)

In [31]:
add_to_dynamodb("onslaught89@gmail.com", "Base")

Item updated successfully: {'ResponseMetadata': {'RequestId': '8V9O1UPOMBV3VKSV1D2MC2UGBVVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 04 May 2024 12:15:53 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '8V9O1UPOMBV3VKSV1D2MC2UGBVVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


Cognito

In [44]:
poolId = "eu-south-1_NTg3Uh7jg"
password = "test123test123!"
birthdate = "01/01/1999"
group = "physio"
prefix="+39"

In [45]:
client = boto3.client('cognito-idp', region_name="eu-south-1")


response = client.admin_create_user(
    UserPoolId=poolId,
    Username=email,
    TemporaryPassword= password,
    UserAttributes=[{"Name": "email","Value": email},  #see them here: https://docs.aws.amazon.com/cognito/latest/developerguide/user-pool-settings-attributes.html
                    {"Name": "email_verified", "Value": "true"},
                    {"Name": "family_name", "Value": first_name},
                    {"Name": "name", "Value": last_name},
                    {"Name": "phone_number", "Value": prefix+phone},
                    {"Name": "birthdate", "Value": birthdate},
                    {"Name": "address", "Value": address}
    ]
)


In [42]:

reply = client.admin_add_user_to_group( UserPoolId=poolId, Username=email, GroupName=group )